In [33]:
import requests, os
from dotenv import load_dotenv
from flask import Flask, request, jsonify
load_dotenv()
mongo_uri = os.getenv("MONGO_URI")
api_key = os.getenv("USDA_API_KEY")

In [45]:
def search_item(query, allWords=False, pageNumber=1, pageSize=20, DataType=None, format=None):
    base_url = "https://api.nal.usda.gov/fdc/v1/foods/search"
    params = {
        "query": query,
        "api_key": "6Rfmv8d36afO9Ptmf0PU79WV22ZdnSaFZi5zshaO",
        "pageSize": pageSize,
        "pageNumber": pageNumber,
        "requireAllWords": allWords,
        "DataType": DataType,
        "format": format
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        if data.get('totalHits') == 0:
            return None

        foods = data.get('foods', [])

        search_criteria = [
            data['foodSearchCriteria'].get('query'),
            data['foodSearchCriteria'].get('pageNumber'),
            data['foodSearchCriteria'].get('pageSize'),
            data['foodSearchCriteria'].get('requireAllWords'),
            data.get('currentPage'),
            data.get('totalPages')
        ]

        return foods, search_criteria
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return False

In [47]:
ty, res = search_item("banana")

branded_items = []
survey_items = []
sr_legacy_items = []
foundation_items = []

for item in ty:
    fdc_id = item.get('fdcId')
    description = item.get('description')
    data_type = item.get('dataType')
    food_nutrients = item.get('foodNutrients', [])
    
    common_info = {
        'fdcId': fdc_id,
        'description': description,
        'dataType': data_type,
        'foodNutrients': food_nutrients
    }
    
    if data_type == 'Branded':
        branded_items.append({
            **common_info,
            'brandOwner': item.get('brandOwner'),
            'brandName': item.get('brandName'),
            'ingredients': item.get('ingredients'),
            'servingSize': item.get('servingSize'),
            'servingSizeUnit': item.get('servingSizeUnit'),
            'householdServingFullText': item.get('householdServingFullText'),
            'gtinUpc': item.get('gtinUpc'),
            'foodCategory': item.get('foodCategory'),
            'packageWeight': item.get('packageWeight'),
            'publishedDate': item.get('publishedDate'),
            'modifiedDate': item.get('modifiedDate')
        })
    elif data_type == 'Survey (FNDDS)':
        survey_items.append({
            **common_info,
            'additionalDescriptions': item.get('additionalDescriptions'),
            'foodCategory': item.get('foodCategory'),
            'foodCode': item.get('foodCode'),
            'publishedDate': item.get('publishedDate')
        })
    elif data_type == 'SR Legacy':
        sr_legacy_items.append({
            **common_info,
            'scientificName': item.get('scientificName'),
            'foodCategory': item.get('foodCategory'),
            'publishedDate': item.get('publishedDate'),
            'ndbNumber': item.get('ndbNumber')
        })
    elif data_type == 'Foundation':
        foundation_items.append({
            **common_info,
            'scientificName': item.get('scientificName'),
            'foodCategory': item.get('foodCategory'),
            'publishedDate': item.get('publishedDate'),
            'ndbNumber': item.get('ndbNumber'),
            'mostRecentAcquisitionDate': item.get('mostRecentAcquisitionDate')
        })
    else:
        print(f"Unknown data type: {data_type}")

def print_items(items):
    for item in items:
        print(f"fdcId: {item['fdcId']}")
        print(f"description: {item['description']}")
        print(f"dataType: {item['dataType']}")
        print("foodNutrients:")
        for nutrient in item['foodNutrients']:
            print(f"  - {nutrient}")
        print("Additional Information:")
        for key, value in item.items():
            if key not in ['fdcId', 'description', 'dataType', 'foodNutrients']:
                print(f"  {key}: {value}")
        print("\n")

# Print all sorted items
print("Branded Items:")
print_items(branded_items)
print("Survey (FNDDS) Items:")
print_items(survey_items)
print("SR Legacy Items:")
print_items(sr_legacy_items)
print("Foundation Items:")
print_items(foundation_items)

Branded Items:
fdcId: 2012128
description: BANANA
dataType: Branded
foodNutrients:
  - {'nutrientId': 1003, 'nutrientName': 'Protein', 'nutrientNumber': '203', 'unitName': 'G', 'derivationCode': 'LCCS', 'derivationDescription': 'Calculated from value per serving size measure', 'derivationId': 70, 'value': 12.5, 'foodNutrientSourceId': 9, 'foodNutrientSourceCode': '12', 'foodNutrientSourceDescription': "Manufacturer's analytical; partial documentation", 'rank': 600, 'indentLevel': 1, 'foodNutrientId': 26035654}
  - {'nutrientId': 1004, 'nutrientName': 'Total lipid (fat)', 'nutrientNumber': '204', 'unitName': 'G', 'derivationCode': 'LCCS', 'derivationDescription': 'Calculated from value per serving size measure', 'derivationId': 70, 'value': 6.25, 'foodNutrientSourceId': 9, 'foodNutrientSourceCode': '12', 'foodNutrientSourceDescription': "Manufacturer's analytical; partial documentation", 'rank': 800, 'indentLevel': 1, 'foodNutrientId': 26035655, 'percentDailyValue': 3}
  - {'nutrientId'